## 1. 패키지 로드

In [1]:
!which python

/home/ubuntu/anaconda3/envs/pytorch_p38/bin/python


In [1]:
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    tf.config.experimental.set_memory_growth(gpus[0], True)
  except RuntimeError as e:
    # 프로그램 시작시에 메모리 증가가 설정되어야만 합니다
    print(e)

In [2]:
'''공통'''

import numpy as np
import pandas as pd

from tqdm import tqdm, tqdm_notebook
import tqdm

'''GPT-2'''
#import tensorflow as tf
from transformers import TFGPT2LMHeadModel

'''KoBERT'''
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp

from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

## 5. GPT3 로드

In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM 

In [4]:
# 토크나이저 로드
gpt3_tokenizer = AutoTokenizer.from_pretrained(
  'kakaobrain/kogpt', revision='KoGPT6B-ryan1.5b-float16',  # or float32 version: revision=KoGPT6B-ryan1.5b
  bos_token='[BOS]', eos_token='[EOS]', unk_token='[UNK]', pad_token='[PAD]', mask_token='[MASK]'
)

In [ ]:
%%time
gpt3_model = AutoModelForCausalLM.from_pretrained(
  'kakaobrain/kogpt', revision='KoGPT6B-ryan1.5b-float16',  # or float32 version: revision=KoGPT6B-ryan1.5b
  pad_token_id=gpt3_tokenizer.eos_token_id,
  torch_dtype='auto', low_cpu_mem_usage=True
).to(device='cuda', non_blocking=True)

In [9]:
_ = gpt3_model.eval()

## 2. 파인튜닝된 KoBERT 로드 

In [14]:
# 학습 데이터 전처리 클래스
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [15]:
# KoBERT 모델 클래스(분류기)
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=4,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [16]:
# vocabulary 불러오기
_, vocab = get_pytorch_kobert_model()

using cached model. /home/lab06/세미프로젝트/GPT3 챗봇/ChatBot_GPT3 & 2/.cache/kobert_v1.zip
using cached model. /home/lab06/세미프로젝트/GPT3 챗봇/ChatBot_GPT3 & 2/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [20]:
### 모델 로드 코드
model_kobert = torch.load('./model/KoBERT_cls_model.pt')
model_kobert.eval()

RuntimeError: CUDA out of memory. Tried to allocate 24.00 MiB (GPU 0; 14.56 GiB total capacity; 11.74 GiB already allocated; 18.44 MiB free; 11.78 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [19]:
torch.cuda.empty_cache()

In [ ]:
#GPU 사용
device = torch.device("cuda:0")

In [ ]:
# params 설정
max_len = 64
batch_size = 64

In [ ]:
# 토크나이저 로드
tokenizer_kobert = get_tokenizer()
tok_kobert = nlp.data.BERTSPTokenizer(tokenizer_kobert, vocab, lower=False)

In [ ]:
def predict_emotion(predict_sentence):
    
    result_emo = None
    
    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok_kobert, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    #model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model_kobert(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("분노가")
                result_emo = '분노'
            elif np.argmax(logits) == 1:
                test_eval.append("행복이")
                result_emo = '행복'
            elif np.argmax(logits) == 2:
                test_eval.append("슬픔이")
                result_emo = '슬픔'
                
        print(">> 입력하신 내용에서 " + test_eval[0] + " 느껴집니다.")
    
    return result_emo

## 3. 파인튜닝된 GPT2 (제목생성) 로드

In [11]:
# GPT2 토크나이저 로드
gpt2_tokenizer = AutoTokenizer.from_pretrained('skt/kogpt2-base-v2', bos_token='</s>', eos_token='</s>', pad_token='<pad>')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [12]:
# gpt2_title_model 로드
gpt2_title_model = TFGPT2LMHeadModel.from_pretrained('./model/Gen_title_GPT2_model.h5')

2022-06-22 10:12:57.228733: E tensorflow/stream_executor/cuda/cuda_blas.cc:232] failed to create cublas handle: CUBLAS_STATUS_NOT_INITIALIZED
2022-06-22 10:12:57.228781: E tensorflow/stream_executor/cuda/cuda_blas.cc:234] Failure to initialize cublas may be due to OOM (cublas needs some free memory when you initialize it, and your deep-learning framework may have preallocated more than its fair share), or may be because this binary was not built with support for the GPU in your machine.
2022-06-22 10:12:57.230380: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at matmul_op_impl.h:438 : INTERNAL: Attempting to perform BLAS operation using StreamExecutor without BLAS support


InternalError: Exception encountered when calling layer "c_attn" (type TFConv1D).

Attempting to perform BLAS operation using StreamExecutor without BLAS support [Op:MatMul]

Call arguments received by layer "c_attn" (type TFConv1D):
  • x=tf.Tensor(shape=(3, 5, 768), dtype=float32)

In [ ]:
def gen_title(user_emotion):
    sent = '<usr>' + user_emotion + '<sys>'
    input_ids = [gpt2_tokenizer.bos_token_id] + gpt2_tokenizer.encode(sent)
    input_ids = tf.convert_to_tensor([input_ids])
    output = gpt2_title_model.generate(input_ids, max_length=50, do_sample=True, top_k=20)
    sentence = gpt2_tokenizer.decode(output[0].numpy().tolist())
    gened_title = sentence.split('<sys> ')[1].replace('</s>', '')
    return gened_title

## 4. 파인튜닝된 GPT2 (첫문장생성) 로드

In [ ]:
gpt2_sent_model = TFGPT2LMHeadModel.from_pretrained('./model/Gen_sent_GPT2_model.h5')

In [ ]:
def gen_sent(user_emotion):
    sent = '<usr>' + user_emotion + '<sys>'
    input_ids = [gpt2_tokenizer.bos_token_id] + gpt2_tokenizer.encode(sent)
    input_ids = tf.convert_to_tensor([input_ids])
    output = gpt2_sent_model.generate(input_ids, max_length=50, do_sample=True, top_k=20)
    sentence = gpt2_tokenizer.decode(output[0].numpy().tolist())
    gened_sent = sentence.split('<sys> ')[1].replace('</s>', '')
    return gened_sent

In [ ]:
def gen_sosal(prmt_title, prmt_sent):
    prompt = f'''
제목 : {prmt_title}
내용 : {prmt_sent}
    '''
    
    with torch.no_grad():
        tokens = gpt3_tokenizer.encode(prompt, return_tensors='pt').to(device='cuda', non_blocking=True)
        gen_tokens = gpt3_model.generate(tokens, do_sample=True, temperature=0.85, max_length=512)
        generated = gpt3_tokenizer.batch_decode(gen_tokens)[0]
    
    return prmt_sent + generated[len(prmt_sent):]

## 6. 소설생성

In [ ]:
# 1. 사용자 입력
user_text = "오늘 너무 행복해~~!"

In [ ]:
# 2. 사용자 입력 감정 분석
user_emotion = predict_emotion(user_text)
user_emotion

In [ ]:
# 3. 제목 생성
user_title = gen_title(user_emotion)
user_title

In [ ]:
# 4. 첫 문장 생성
user_sent = gen_sent(user_emotion)
user_sent

In [ ]:
# 5. 소설 생성
sosal_content = gen_sosal(user_title, user_sent)

In [ ]:
print(f'''
제목 : {user_title}
내용 : {sosal_content}
''')